In [1]:
!pip install evalml

     |████████████████████████████████| 6.4 MB 5.2 MB/s 
     |████████████████████████████████| 38.1 MB 1.3 MB/s 
     |████████████████████████████████| 79.9 MB 117 kB/s 
     |████████████████████████████████| 18.0 MB 523 kB/s 
     |████████████████████████████████| 275 kB 51.3 MB/s 
     |████████████████████████████████| 1.4 MB 52.7 MB/s 
     |████████████████████████████████| 173.6 MB 7.2 kB/s 
     |████████████████████████████████| 339 kB 58.4 MB/s 
     |████████████████████████████████| 15.7 MB 25.4 MB/s 
     |████████████████████████████████| 9.8 MB 36.1 MB/s 
     |████████████████████████████████| 2.0 MB 52.0 MB/s 
     |████████████████████████████████| 1.6 MB 54.9 MB/s 
     |████████████████████████████████| 82 kB 361 kB/s 
     |████████████████████████████████| 160 kB 45.8 MB/s 
     |████████████████████████████████| 11.2 MB 41.3 MB/s 
     |████████████████████████████████| 46 kB 3.5 MB/s 
     |████████████████████████████████| 3.0 MB 59.3 MB/s 
     |██████████

In [3]:
import evalml
import pandas as pd
import numpy as np

In [4]:
data = pd.read_csv('mwc22-client_table.csv')

In [5]:
data.head()

,CLIENT ID,CLIENT_SEGMENT,AVG CONSO,AVG BASKET SIZE,RECEIVED_COMMUNICATION
0,24321771,6,"67,25","201,75",0
1,24321859,2,"58,33",350,0
2,24321880,3,"46,67",112,0
3,24321957,2,50,600,0
4,24321962,4,10,120,0


In [16]:
data2 = data.copy()

In [18]:
data2['AVG CONSO'] = data2['AVG CONSO'].astype(str)
data2['AVG BASKET SIZE'] = data2["AVG BASKET SIZE"].astype(str)
data2['AVG CONSO']=data2['AVG CONSO'].apply(lambda x: x.replace(',','.'))
data2['AVG BASKET SIZE'] = data2["AVG BASKET SIZE"].apply(lambda x: x.replace(',','.'))
data2['AVG CONSO'] = data2['AVG CONSO'].astype(float)
data2['AVG BASKET SIZE'] = data2["AVG BASKET SIZE"].astype(float)
data2['CLIENT ID']=data2['CLIENT ID'].astype(str)

In [22]:
data2.describe()

,CLIENT_SEGMENT,AVG CONSO,AVG BASKET SIZE,RECEIVED_COMMUNICATION
count,35884.000000,35884.000000,35884.000000,35884.000000
mean,3.123593,64.534437,181.218610,0.507998
std,1.512932,64.381932,129.604691,0.499943
min,1.000000,0.830000,10.000000,0.000000
25%,2.000000,20.830000,100.000000,0.000000
50%,3.000000,50.000000,160.000000,1.000000
75%,4.000000,88.170000,225.000000,1.000000
max,6.000000,2433.330000,3400.200000,1.000000


In [30]:
data2.isnull().sum()

CLIENT ID                 0
CLIENT_SEGMENT            0
AVG CONSO                 0
AVG BASKET SIZE           0
RECEIVED_COMMUNICATION    0
dtype: int64

In [23]:
X = data2.drop('CLIENT_SEGMENT', axis = 1)
y = data2['CLIENT_SEGMENT']

In [24]:
X_train, X_test, y_train, y_test = evalml.preprocessing.split_data(X, y, problem_type='multiclass')

In [25]:
from evalml.automl import AutoMLSearch
automl = AutoMLSearch(X_train=X_train, y_train=y_train, problem_type='multiclass')
automl.search()

In [26]:
automl.rankings

,id,pipeline_name,search_order,mean_cv_score,standard_deviation_cv_score,validation_score,percent_better_than_baseline,high_variance_cv,parameters
0,3,XGBoost Classifier w/ Label Encoder + Drop Col...,3,0.569292,4.579455e-03,0.569292,97.869475,False,"{'Label Encoder': {'positive_label': None}, 'D..."
1,4,LightGBM Classifier w/ Label Encoder + Drop Co...,4,0.575444,5.016046e-03,0.575444,97.846449,False,"{'Label Encoder': {'positive_label': None}, 'D..."
2,6,Random Forest Classifier w/ Label Encoder + Dr...,6,0.662792,4.780718e-03,0.662792,97.519560,False,"{'Label Encoder': {'positive_label': None}, 'D..."
3,7,Decision Tree Classifier w/ Label Encoder + Dr...,7,0.849360,1.920268e-02,0.849360,96.821342,False,"{'Label Encoder': {'positive_label': None}, 'D..."
4,2,Logistic Regression Classifier w/ Label Encode...,2,0.867519,8.928185e-03,0.867519,96.753384,False,"{'Label Encoder': {'positive_label': None}, 'D..."
5,1,Elastic Net Classifier w/ Label Encoder + Drop...,1,0.880853,8.869558e-03,0.880853,96.703485,False,"{'Label Encoder': {'positive_label': None}, 'D..."
6,8,Extra Trees Classifier w/ Label Encoder + Drop...,8,1.268096,1.173190e-02,1.268096,95.254258,False,"{'Label Encoder': {'positive_label': None}, 'D..."
7,5,CatBoost Classifier w/ Label Encoder + Drop Co...,5,1.342418,2.947907e-03,1.342418,94.976117,False,"{'Label Encoder': {'positive_label': None}, 'D..."
8,0,Mode Baseline Multiclass Classification Pipeline,0,26.720719,2.512148e-15,26.720719,0.000000,False,"{'Label Encoder': {'positive_label': None}, 'B..."


In [27]:
automl.best_pipeline

pipeline = MulticlassClassificationPipeline(component_graph={'Label Encoder': ['Label Encoder', 'X', 'y'], 'Drop Columns Transformer': ['Drop Columns Transformer', 'X', 'Label Encoder.y'], 'Imputer': ['Imputer', 'Drop Columns Transformer.x', 'Label Encoder.y'], 'XGBoost Classifier': ['XGBoost Classifier', 'Imputer.x', 'Label Encoder.y']}, parameters={'Label Encoder':{'positive_label': None}, 'Drop Columns Transformer':{'columns': ['CLIENT ID']}, 'Imputer':{'categorical_impute_strategy': 'most_frequent', 'numeric_impute_strategy': 'mean', 'categorical_fill_value': None, 'numeric_fill_value': None}, 'XGBoost Classifier':{'eta': 0.1, 'max_depth': 6, 'min_child_weight': 1, 'n_estimators': 100, 'n_jobs': -1, 'eval_metric': 'logloss'}}, random_seed=0)

In [28]:
best_pipeline=automl.best_pipeline

In [34]:
automl_auc = AutoMLSearch(X_train=X_train, y_train=y_train,
                          problem_type='multiclass',
                          max_batches=1,
                          optimize_thresholds=True)

automl_auc.search()

In [35]:
automl_auc.rankings

,id,pipeline_name,search_order,mean_cv_score,standard_deviation_cv_score,validation_score,percent_better_than_baseline,high_variance_cv,parameters
0,3,XGBoost Classifier w/ Label Encoder + Drop Col...,3,0.569292,4.579455e-03,0.569292,97.869475,False,"{'Label Encoder': {'positive_label': None}, 'D..."
1,4,LightGBM Classifier w/ Label Encoder + Drop Co...,4,0.575444,5.016046e-03,0.575444,97.846449,False,"{'Label Encoder': {'positive_label': None}, 'D..."
2,6,Random Forest Classifier w/ Label Encoder + Dr...,6,0.662792,4.780718e-03,0.662792,97.519560,False,"{'Label Encoder': {'positive_label': None}, 'D..."
3,7,Decision Tree Classifier w/ Label Encoder + Dr...,7,0.849360,1.920268e-02,0.849360,96.821342,False,"{'Label Encoder': {'positive_label': None}, 'D..."
4,2,Logistic Regression Classifier w/ Label Encode...,2,0.867519,8.928185e-03,0.867519,96.753384,False,"{'Label Encoder': {'positive_label': None}, 'D..."
5,1,Elastic Net Classifier w/ Label Encoder + Drop...,1,0.880853,8.869558e-03,0.880853,96.703485,False,"{'Label Encoder': {'positive_label': None}, 'D..."
6,8,Extra Trees Classifier w/ Label Encoder + Drop...,8,1.268096,1.173190e-02,1.268096,95.254258,False,"{'Label Encoder': {'positive_label': None}, 'D..."
7,5,CatBoost Classifier w/ Label Encoder + Drop Co...,5,1.342418,2.947907e-03,1.342418,94.976117,False,"{'Label Encoder': {'positive_label': None}, 'D..."
8,0,Mode Baseline Multiclass Classification Pipeline,0,26.720719,2.512148e-15,26.720719,0.000000,False,"{'Label Encoder': {'positive_label': None}, 'B..."


In [36]:
automl_auc.describe_pipeline(automl_auc.rankings.iloc[0]["id"])


****************************************************************************
* XGBoost Classifier w/ Label Encoder + Drop Columns Transformer + Imputer *
****************************************************************************

Problem Type: multiclass
Model Family: XGBoost

Pipeline Steps
1. Label Encoder
	 * positive_label : None
2. Drop Columns Transformer
	 * columns : ['CLIENT ID']
3. Imputer
	 * categorical_impute_strategy : most_frequent
	 * numeric_impute_strategy : mean
	 * categorical_fill_value : None
	 * numeric_fill_value : None
4. XGBoost Classifier
	 * eta : 0.1
	 * max_depth : 6
	 * min_child_weight : 1
	 * n_estimators : 100
	 * n_jobs : -1
	 * eval_metric : logloss

Training
Training for multiclass problems.
Total training time (including CV): 15.7 seconds

Cross Validation
----------------
             Log Loss Multiclass  MCC Multiclass  AUC Weighted  AUC Macro  AUC Micro  Precision Weighted  Precision Macro  Precision Micro  F1 Weighted  F1 Macro  F1 Micro  Ba